In [100]:
import torch
import torch.nn as nn
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd

In [101]:
dataframe = pd.read_csv('diabetes.csv')
X = dataframe.drop(columns='Outcome')
y = dataframe['Outcome']

X_train, X_test, y_train, y_test = train_test_split(X,y,train_size = 0.8, random_state = 42)

dataframe


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [102]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [103]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [104]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).unsqueeze(1).to(device) #need to convert to numpy arrays use .values
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).unsqueeze(1).to(device)

In [105]:
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_dataloader = DataLoader(train_dataset, batch_size= 16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [106]:
class BinaryClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(BinaryClassifier, self).__init__()
        self.layer1 = nn.Linear(input_size, 256)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.7)  # Add dropout
        self.layer2 = nn.Linear(256, hidden_size)
        self.layer3 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.layer1(x)
        x = self.relu(x)
        x = self.dropout(x)  # Apply dropout
        x = self.layer2(x)
        x = self.layer3(x)
        return x  # No sigmoid here, for BCEWithLogitsLoss

input_size = X.shape[1]
hidden_size = 92
output_size = 1 #binary classification

print(f'input Size: {input_size}, Output Size: {output_size}')

input Size: 8, Output Size: 1


In [107]:
model = BinaryClassifier(input_size, hidden_size, output_size).to(device)
criterion = nn.BCEWithLogitsLoss()
optimiser = torch.optim.AdamW(model.parameters(), lr = 0.001)



In [108]:
num_epochs = 1000

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for X_batch, y_batch in train_dataloader:
        optimiser.zero_grad()
        y_predicted = model(X_batch)
        loss = criterion(y_predicted, y_batch)
        loss.backward()
        optimiser.step()
        epoch_loss += loss.item()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss/len(train_dataloader):.4f}')
    

Epoch [100/1000], Loss: 0.4428
Epoch [200/1000], Loss: 0.4108
Epoch [300/1000], Loss: 0.3726
Epoch [400/1000], Loss: 0.3751
Epoch [500/1000], Loss: 0.3518
Epoch [600/1000], Loss: 0.3063
Epoch [700/1000], Loss: 0.3023
Epoch [800/1000], Loss: 0.3123
Epoch [900/1000], Loss: 0.3085
Epoch [1000/1000], Loss: 0.3052


In [109]:
# Evaluate the model
model.eval()
with torch.no_grad():
    all_preds = []
    all_labels = []
    for X_batch, y_batch in test_dataloader:
        y_predicted = model(X_batch)
        y_predicted = torch.sigmoid(y_predicted)  # Apply sigmoid to get probabilities
        all_preds.append(y_predicted)
        all_labels.append(y_batch)

    y_predicted = torch.cat(all_preds)
    y_labels = torch.cat(all_labels)
    y_predicted = (y_predicted > 0.5).float()  # Convert probabilities to binary predictions

    accuracy = accuracy_score(y_labels.cpu(), y_predicted.cpu())
    print(f'Test Accuracy: {accuracy:.4f}')

Test Accuracy: 0.7143
